# Import libraries

In [8]:
%pip install soilgrids
%pip install xarray
%pip install pyproj
%pip install numpy
%pip install kwargs
import logging
import os
import time
from soilgrids import SoilGrids
import xarray as xr
from pyproj import Transformer
import numpy as np
import kwargs
from owslib.wcs import WebCoverageService



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Class of constants

In [2]:
class Constants:
    '''Pass all constants as class attributes.'''

    def __init__(self):
        '''Initialize Constants object.'''
        self.soil_vars = ['bdod', 'cec', 'cfvo', 'clay',
                          'nitrogen', 'phh2o', 'sand', 'silt', 'soc', 'ocd', 'ocs']
        self.soil_depth = ['0-5', '5-15',
                           '15-30', '30-60', '60-100', '100-200']

# Functions

In [3]:
def move_point(latitude, longitude, distance, bearing):
    earth_radius = 6371000  # Earth's radius in meters
    angular_distance = distance / earth_radius
    # in degrees, 0 is north, 90 is east, 180 is south, 270 is west
    bearing = math.radians(bearing)

    lat1 = math.radians(latitude)
    lon1 = math.radians(longitude)

    lat2 = math.asin(math.sin(lat1) * math.cos(angular_distance) +
                     math.cos(lat1) * math.sin(angular_distance) * math.cos(bearing))
    lon2 = lon1 + math.atan2(math.sin(bearing) * math.sin(angular_distance) * math.cos(lat1),
                             math.cos(angular_distance) - math.sin(lat1) * math.sin(lat2))

    new_latitude = math.degrees(lat2)
    new_longitude = math.degrees(lon2)

    return new_latitude, new_longitude

In [9]:
soilvars = Constants().soil_vars
soildepth = Constants().soil_depth
var = soilvars[0]
wcs = WebCoverageService(
    f'http://maps.isric.org/mapserv?map=/map/{var}.map', version='2.0.1')

In [10]:
subsets = [('X', -1784000, -1140000), ('Y', 1356000, 1863000)]
crs = "http://www.opengis.net/def/crs/EPSG/0/152160"

## On demand download

In [2]:
def on_demand_download(geo_type: str, geo_code0: float, geo_code1: float):
    '''On-demand download SoilGrids data for a given geo-code.'''

    if geo_type == 'CRS':
        west, east, south, north = geo_code0 - 0.5e6, geo_code0 + \
            0.5e6, geo_code1 - 0.8e6, geo_code1 + 0.8e6
        transformer = Transformer.from_crs(27700, 4326, always_xy=True)
        bl = transformer.transform(west, south)
        br = transformer.transform(east, south)
        tl = transformer.transform(west, north)
        tr = transformer.transform(east, north)
    elif geo_type == 'lat-lon':
        transformer = Transformer.from_crs(4326, 27700, always_xy=True)
        x1, y1 = transformer.transform(geo_code0, geo_code1)
        west, east, south, north = x1 - 0.5e6, x1 + 0.5e6, y1 - 0.8e6, y1 + 0.8e6
        transformer = Transformer.from_crs(27700, 4326, always_xy=True)
        bl = transformer.transform(west, south)
        br = transformer.transform(east, south)
        tl = transformer.transform(west, north)
        tr = transformer.transform(east, north)
    else:
        print('Invalid geo_type. Please enter a valid geo_type("CRS" or "lat-lon").')
        return

    # get soil data
    soil_grids = SoilGrids()

    # Conversion functions
    def NoConversion(x): return x
    def mult_hundred(x): return x*100.
    def mult_ten(x): return x*10.
    def div_ten(x): return x/10.
    def div_hundred(x): return x/100.

    # Conversion by soil parameter (see https://www.isric.org/explore/soilgrids/faq-soilgrids)
    obs_conversions = {
        "bdod": mult_hundred,
        "cec": div_ten,
        "cfvo": div_ten,
        "clay": div_ten,
        "nitrogen": div_hundred,
        "phh2o": div_ten,
        "sand": div_ten,
        "silt": div_ten,
        "soc": div_ten,
        "ocd": div_ten,
        "ocs": mult_ten
    }

    soilvars = Constants().soil_vars
    soil_depths = Constants().soil_depth

    counter = 1
    soil_data = xr.Dataset()

    for var in soilvars:
        print(
            f'Processing variable {counter} of {len(soilvars)}: \'{var}\'')
        soil_chunk = xr.Dataset()
        if var == 'ocs':
            while True:  # required because the server sometimes returns a 500 error
                depth = '0-30'
                try:
                    soil_data[var] = soil_grids.get_coverage_data(service_id=var, coverage_id=f'{var}_{depth}cm_mean',
                                                                  west=bl[0], south=bl[1], east=br[0], north=tr[1],
                                                                  width=4000, height=6400,
                                                                  crs='urn:ogc:def:crs:EPSG::4326', output='tmp.tif')
                    # conversion to conventional units
                    func = obs_conversions[var]
                    soil_data[var] = func(soil_data[var])
                except:
                    print("get_coverage_data failed. Retrying in 60 seconds...")
                    time.sleep(60)
                    continue
                break
        else:
            for depth in soil_depths:
                while True:
                    try:
                        soil_chunk[depth] = soil_grids.get_coverage_data(service_id=var, coverage_id=f'{var}_{depth}cm_mean',
                                                                         west=bl[0], south=bl[1], east=br[0], north=tr[1],
                                                                         width=4000, height=6400,
                                                                         crs='urn:ogc:def:crs:EPSG::4326', output=('tmp.tif'))
                        # conversion to conventional units
                        func = obs_conversions[var]
                        soil_chunk[depth] = func(soil_chunk[depth])
                    except:
                        print("get_coverage_data failed. Retrying in 60 seconds...")
                        time.sleep(60)
                        continue
                    break

            # Weighted average of soil variables for depths up to 60cm
            weight_factor = {
                '0-5': 1,
                '5-15': 2,
                '15-30': 3,
                '30-60': 6
            }

            # (i.e. xr.DataArray()) because dimensions cannot change for in-place operations
            soil_data[var] = soil_chunk[depth] * 0

            for key in weight_factor.keys():
                df = soil_chunk[key] * weight_factor[key]
                soil_data[var] += df

            soil_data[var] = soil_data[var] / 12
            soil_data[var] = soil_data[var].where(
                soil_data[var] != 0, np.nan)
            counter += 1
            time.sleep(60)

    return soil_data

## Bulk download

In [3]:
def bulk_download(xmin: float, xmax: float, ymin: float, ymax: float):
    '''bulk download SoilGrids data for a given boundary.'''

    '''
    if geo_type == 'CRS':
        west, east, south, north = geo_code0 - 0.5e6, geo_code0 + 0.5e6, geo_code1 - 0.8e6, geo_code1 + 0.8e6
        transformer = Transformer.from_crs(27700, 4326, always_xy=True)
        bl = transformer.transform(west, south)
        br = transformer.transform(east, south)
        tl = transformer.transform(west, north)
        tr = transformer.transform(east, north)
    elif geo_type == 'lat-lon':
        transformer = Transformer.from_crs(4326, 27700, always_xy=True)
        x1, y1 = transformer.transform(geo_code0, geo_code1)
        west, east, south, north = x1 - 0.5e6, x1 + 0.5e6, y1 - 0.8e6, y1 + 0.8e6
        transformer = Transformer.from_crs(27700, 4326, always_xy=True)
        bl = transformer.transform(west, south)
        br = transformer.transform(east, south)
        tl = transformer.transform(west, north)
        tr = transformer.transform(east, north)
    else:
        raise ValueError('Invalid geo_type. Please enter a valid geo_type("CRS" or "lat-lon").')
    '''
    # get soil data
    soil_grids = SoilGrids()

    # Conversion functions
    def NoConversion(x): return x
    def mult_hundred(x): return x*100.
    def mult_ten(x): return x*10.
    def div_ten(x): return x/10.
    def div_hundred(x): return x/100.

    # Conversion by soil parameter (see https://www.isric.org/explore/soilgrids/faq-soilgrids)
    obs_conversions = {
        "bdod": mult_hundred,
        "cec": div_ten,
        "cfvo": div_ten,
        "clay": div_ten,
        "nitrogen": div_hundred,
        "phh2o": div_ten,
        "sand": div_ten,
        "silt": div_ten,
        "soc": div_ten,
        "ocd": div_ten,
        "ocs": mult_ten}

    soilvars = Constants().soil_vars
    soil_depths = Constants().soil_depth

    counter = 1
    soil_data = xr.Dataset()
    for var in soilvars:
        print(
            f'Processing variable {counter} of {len(soilvars)}: \'{var}\'')
        soil_chunk = xr.Dataset()
        if var == 'ocs':
            while True:  # required because the server sometimes returns a 500 error
                depth = '0-30'
                try:
                    soil_data[var] = soil_grids.get_coverage_data(service_id=var, coverage_id=f'{var}_{depth}cm_mean',
                                                                  west=xmin, south=ymin, east=xmax, north=ymax,
                                                                  width=4000, height=6400,
                                                                  crs='urn:ogc:def:crs:EPSG::4326', output=('tmp.tif'))
                    # conversion to conventional units
                    func = obs_conversions[var]
                    soil_data[var] = func(soil_data[var])
                except:
                    print("get_coverage_data failed. Retrying in 60 seconds...")
                    time.sleep(60)
                    continue
                break
        else:
            for depth in soil_depths:
                import requests.exceptions
                while True:
                    try:
                        soil_chunk[depth] = soil_grids.get_coverage_data(service_id=var, coverage_id=f'{var}_{depth}cm_mean',
                                                                         west=xmin, south=ymin, east=xmax, north=ymax,
                                                                         width=4000, height=6400,
                                                                         crs='urn:ogc:def:crs:EPSG::4326', output=('tmp.tif'))
                        # conversion to conventional units
                        func = obs_conversions[var]
                        soil_chunk[depth] = func(soil_chunk[depth])
                    except:
                        print("get_coverage_data failed. Retrying in 60 seconds...")
                        time.sleep(60)
                        continue
                    break

            # Weighted average of soil variables for depths up to 60cm
            weight_factor = {
                '0-5': 1,
                '5-15': 2,
                '15-30': 3,
                '30-60': 6
            }
            # empty array to store the weighted average. Can't be an empty object
            # (i.e. xr.DataArray()) because dimensions cannot change for in-place operations
            soil_data[var] = soil_chunk[depth] * 0

            for key in weight_factor.keys():
                df = soil_chunk[key] * weight_factor[key]
                soil_data[var] += df

            soil_data[var] = soil_data[var] / 12
            soil_data[var] = soil_data[var].where(
                soil_data[var] != 0, np.nan)
            counter += 1
            time.sleep(60)

    soil_data.to_netcdf('GB_soil_data.nc')
    return soil_data

# Test functions

In [4]:
transformer = Transformer.from_crs(4326, 27700, always_xy=True)
# centroid of Great Britain
x1, y1 = transformer.transform(-2.547855, 54.00366)
west, east, south, north = x1 - 0.5e6, x1 + 0.5e6, y1 - 0.8e6, y1 + 0.8e6
transformer = Transformer.from_crs(27700, 4326, always_xy=True)
bl = transformer.transform(west, south)
br = transformer.transform(east, south)
tl = transformer.transform(west, north)
tr = transformer.transform(east, north)

In [5]:
data = on_demand_download('lat-lon', -2.547855, 54.00366)
print(data)

Processing variable 1 of 11: 'bdod'
Processing variable 2 of 11: 'cec'
Processing variable 3 of 11: 'cfvo'
Processing variable 4 of 11: 'clay'
Processing variable 5 of 11: 'nitrogen'
Processing variable 6 of 11: 'phh2o'
get_coverage_data failed. Retrying in 60 seconds...
Processing variable 7 of 11: 'sand'
get_coverage_data failed. Retrying in 60 seconds...
Processing variable 8 of 11: 'silt'
Processing variable 9 of 11: 'soc'
get_coverage_data failed. Retrying in 60 seconds...
Processing variable 10 of 11: 'ocd'
get_coverage_data failed. Retrying in 60 seconds...
Processing variable 11 of 11: 'ocs'
<xarray.Dataset>
Dimensions:      (band: 1, x: 4000, y: 6400)
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -8.995 -8.992 -8.989 -8.985 ... 4.059 4.062 4.066
  * y            (y) float64 60.92 60.91 60.91 60.91 ... 46.6 46.6 46.6 46.6
    spatial_ref  int64 0
Data variables:
    bdod         (band, y, x) float64 nan nan nan nan ... nan nan 1.46e+04 nan
    cec   

In [6]:
bulk_data = bulk_download(bl[0], bl[1], tr[0], tl[1])
print(bulk_data)

Processing variable 1 of 11: 'bdod'
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Retrying in 60 seconds...
get_coverage_data failed. Re